In [1]:
%env CUDA_VISIBLE_DEVICES=MIG-cbafb023-40ef-594e-9092-fb0e3c44baa2

env: CUDA_VISIBLE_DEVICES=MIG-cbafb023-40ef-594e-9092-fb0e3c44baa2


In [3]:
import io
import time
import torch
import torch.nn as nn
import PIL.Image
import io
import numpy as np
import datasets
from torchvision.transforms.v2 import Pad, CenterCrop
from torchvision.transforms.v2.functional import pil_to_tensor, to_pil_image
from types import SimpleNamespace
from autocodec.codec import AutoCodecND, latent_to_pil, pil_to_latent
from piq import LPIPS, DISTS, SSIMLoss

In [4]:
device = "cuda"
lpips_loss = LPIPS().to(device)
dists_loss = DISTS().to(device)
ssim_loss = SSIMLoss().to(device)

/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dgj335/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
inet = datasets.load_dataset("timm/imagenet-1k-wds",split='validation')

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [21]:
checkpoint = torch.load('../../hf/dance/rgb_f8c12.pth', map_location=device,weights_only=False)
config = checkpoint['config']
state_dict = checkpoint['state_dict']

model = AutoCodecND(
    dim=2,
    input_channels=config.input_channels,
    J = int(np.log2(config.F)),
    latent_dim=config.latent_dim,
    encoder_depth = config.encoder_depth,
    encoder_kernel_size = config.encoder_kernel_size,
    decoder_depth = config.decoder_depth,
    lightweight_encode = config.lightweight_encode,
    lightweight_decode = config.lightweight_decode,
).to(device)
model.load_state_dict(state_dict)
model.eval();

In [17]:
def compress_and_evaluate(sample):
    img = sample['jpg'].convert("RGB")
    aspect = img.width/img.height
    img = img.resize((int(16*(1024*aspect//16)),1024),resample=PIL.Image.Resampling.LANCZOS)
    x_orig = pil_to_tensor(img).to(device).unsqueeze(0).to(torch.float) / 127.5 - 1.0
    orig_dim = x_orig.numel() 

    t0 = time.time()
    with torch.no_grad():
        z = model.encode(x_orig)
        latent = model.quantize.compand(z).round()
    webp = latent_to_pil(latent.cpu(), n_bits=8, C=3)
    buff = io.BytesIO()
    webp[0].save(buff, format='WEBP', lossless=True)
    encode_time = time.time() - t0
    size_bytes = len(buff.getbuffer())
    t0 = time.time()
    latent_decoded = pil_to_latent(webp, N=config.latent_dim, n_bits=8, C=3).to(device)
    with torch.no_grad():
        x_hat = model.decode(latent_decoded).clamp(-1,1)
    decode_time = time.time() - t0

    x_orig_01 = x_orig / 2 + 0.5
    x_hat_01 = x_hat / 2 + 0.5

    pixels = img.width * img.height
    bpp = 8 * size_bytes / pixels
    mse = torch.nn.functional.mse_loss(x_orig_01[0], x_hat_01[0])
    PSNR = -10 * mse.log10().item()
    LPIPS_dB = -10 * np.log10(lpips_loss(x_orig_01.to(device), x_hat_01.to(device)).item())
    DISTS_dB = -10 * np.log10(dists_loss(x_orig_01.to(device), x_hat_01.to(device)).item())
    SSIM = 1 - ssim_loss(x_orig_01.to(device), x_hat_01.to(device)).item()

    return {
        'pixels': pixels,
        'bpp': bpp,
        'PSNR': PSNR,
        'LPIPS_dB': LPIPS_dB,
        'DISTS_dB': DISTS_dB,
        'SSIM': SSIM,
        'encode_time': encode_time,
        'decode_time': decode_time,
    }

metrics = [
    'pixels',
    'bpp',
    'PSNR',
    'LPIPS_dB',
    'DISTS_dB',
    'SSIM',
    'encode_time',
    'decode_time',
]

f16c12

In [14]:
results_dataset = inet.select(range(100)).map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

mean
---
pixels: 1227161.6
bpp: 0.14648919167678695
PSNR: 31.021453261375427
LPIPS_dB: 5.973899995484275
DISTS_dB: 11.845230856928634
SSIM: 0.9448394024372101
encode_time: 0.015861053466796875
decode_time: 0.008933014869689941
80.04927627255795 MP/sec


f8c12 lsdir

In [22]:
results_dataset = inet.select(range(100)).map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

mean
---
pixels: 1227161.6
bpp: 0.5912512305003398
PSNR: 36.13458340167999
LPIPS_dB: 8.652118808188026
DISTS_dB: 16.805981208572174
SSIM: 0.9812395268678665
encode_time: 0.05376312255859375
decode_time: 0.007639296054840088
26.120858451126278 MP/sec


f8c12 lsdir+inet

In [18]:
results_dataset = inet.select(range(100)).map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

mean
---
pixels: 1227161.6
bpp: 0.6510875973063395
PSNR: 38.71788156032562
LPIPS_dB: 10.73722696227855
DISTS_dB: 19.70042709393359
SSIM: 0.988203484416008
encode_time: 0.057140915393829345
decode_time: 0.00765207052230835
24.919460404154268 MP/sec


f8c12 lsdir+inet+lsdir

In [10]:
results_dataset = inet.select(range(100)).map(compress_and_evaluate)

print("mean\n---")
for metric in metrics:
    μ = np.mean(results_dataset[metric])
    print(f"{metric}: {μ}")
print(f"{np.mean(np.array(results_dataset['pixels'])/1e6/np.array(results_dataset['encode_time']))} MP/sec")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

mean
---
pixels: 1227161.6
bpp: 0.6391400756471314
PSNR: 37.93727173805237
LPIPS_dB: 9.995588076564129
DISTS_dB: 19.153368748923363
SSIM: 0.9857030594348908
encode_time: 0.0602103853225708
decode_time: 0.010009536743164063
23.517455320890708 MP/sec
